### 第三章 指数基金的选择

#### 3.1 ETF300指数基金

In [1]:
import tushare as ts
token = '6feca07b866ea55bd6024aa5a8891d7bc11afd829d1386c8a2fc8d89'
pro = ts.pro_api(token)

In [61]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import datetime
import pandas as pd
import numpy as np
from scipy import stats
import time
import requests
from IPython.core.display import HTML

In [43]:
def combine_fund():
    tf = ['ts_code', 'name', 'management', 'benchmark', 'm_fee', 'c_fee', 'issue_amount', 'fund_type']
    zf = pro.fund_basic(market='E')
    zf_e = zf[tf]
    zf = pro.fund_basic(market='O')
    zf_o = zf[tf]
    zfl = pd.concat([zf_e, zf_o], axis=0)
    return zfl
    
def filter_fund(fund_type, benchmark, zfl):    
    zfl = zfl[zfl['fund_type'].str.contains(fund_type, na=False)]
    zfl = zfl[zfl['benchmark'].str.contains(benchmark, na=False)]
    return zfl

In [46]:
f = '股票型'
b = '沪深300指数'
zf = combine_fund()
zfl = filter_fund(f, b, zf)
zfl[zfl['m_fee']<0.6]

ts_code             name management  \
2      510380.SH            国寿300     国寿安保基金   
4      510390.SH            平安300     平安大华基金   
19     501043.SH           沪深300A      汇添富基金   
132    510360.SH            广发300       广发基金   
389    159927.SZ          A300ETF       鹏华基金   
418    510310.SH         HS300ETF      易方达基金   
422    159925.SZ            南方300       南方基金   
434    510330.SH            华夏300       华夏基金   
475    159919.SZ           300ETF       嘉实基金   
479    510300.SH           300ETF     华泰柏瑞基金   
623   1618111.SZ            银华300       银华基金   
26     006131.OF  华泰柏瑞沪深300ETF联接C     华泰柏瑞基金   
1283   003015.OF         中金沪深300A       中金基金   
3231   110020.OF    易方达沪深300ETF联接      易方达基金   
3242   000051.OF    华夏沪深300ETF联接A       华夏基金   
3268   202015.OF  南方开元沪深300ETF联接A       南方基金   
3275   481009.OF        工银瑞信沪深300     工银瑞信基金   
3282   270010.OF    广发沪深300ETF联接A       广发基金   

                               benchmark  m_fee  c_fee  issue_amount fund_type  
2                                沪深300指数    0.5   0.10        2.9480       股票型  
4                             沪深300指数收益率    0.5   0.10       57.2306       股票型  
19     沪深300指数收益率*95%+银行人民币活期存款利率(税后)*5%    0.5   0.10        2.1586       股票型  
132                           沪深300指数收益率    0.5   0.10        3.0838       股票型  
389                              沪深300指数    0.5   0.10       13.0658       股票型  
418                              沪深300指数    0.2   0.10       11.0431       股票型  
422                              沪深300指数    0.5   0.10       39.5920       股票型  
434                              沪深300指数    0.5   0.10        6.0281       股票型  
475                              沪深300指数    0.5   0.10      193.3331       股票型  
479                              沪深300指数    0.5   0.10      329.6863       股票型  
623     沪深300指数收益率*95%+商业银行活期存款利率(税后)*5%    0.5   0.15        6.7019       股票型  
26          沪深300指数*95%+银行活期存款收益率(税后)*5%    0.5   0.10           NaN       股票型  
1283      沪深300指数收益率*95%+银行活期存款利率(税后)*5%    0.5   0.15        0.1000       股票型  
3231        沪深300指数收益率*95%+活期存款利率(税后)*5%    0.2   0.10      167.4625       股票型  
3242  沪深300指数收益率*95%+1%(指年收益率,评价时应按期间折算)    0.5   0.10      247.7196       股票型  
3268      沪深300指数收益率*95%+银行活期存款利率(税后)*5%    0.5   0.10       15.8075       股票型  
3275    沪深300指数收益率*95%+商业银行活期存款利率(税后)*5%    0.5   0.10       36.0642       股票型  
3282      沪深300指数收益率*95%+银行活期存款利率(税后)*5%    0.5   0.10        6.5369       股票型

#### 3.2 行业指数基金

In [53]:
bs = ['中证能源','中证材料','中证工业','中证可选','中证消费','中证医药','中证金融','中证信息','中证电信','中证公用']
f = '股票型'
zf = combine_fund()
for b in bs:
    zfl = filter_fund(f, b,zf)
    zfl = zfl[zfl['m_fee']<2.0]
    print(b)
    display(HTML(zfl.to_html()))

中证能源


中证材料


中证工业


中证可选


中证消费


中证医药


中证金融


中证信息


中证电信


中证公用


In [60]:
b = '银行'
zfl = filter_fund(f, b,zf)
zfl = zfl[zfl['m_fee']<2.0]
print(b)
display(HTML(zfl.to_html()))

银行


#### 3.3 Track Error

In [140]:
# parameters inited
time_spread = 3 # Years
t = datetime.date.today()
end_date = t
start_date = datetime.datetime(t.year - time_spread, t.month, t.day)
end_date_str = end_date.strftime("%Y%m%d")
start_date_str = start_date.strftime("%Y%m%d")

In [141]:
hs300 = '399300.SZ'
etf300e = '510310.SH'
etf300o = '110020.OF'

In [142]:
df1 = pro.index_daily(ts_code=hs300, start_date=start_date_str, end_date=end_date_str)

In [143]:
df2 = pro.fund_daily(ts_code=etf300e, start_date=start_date_str, end_date=end_date_str)

In [144]:
df3 = pro.fund_nav(ts_code=etf300o) #, start_date=start_date_str, end_date=end_date_str)

In [148]:
def track_error(b,m,market='E'):
    t = ['trade_date', 'close']
    b = b[t]
    b = b.rename({'close':'b'}, axis=1)  
    if (market != 'E'):
        m['close'] = m['unit_nav']
        m['trade_date'] = m['end_date']
    m = m[t]
    m = m.rename({'close':'m'}, axis=1)
    if (len(b)<=len(m)):
        c = pd.merge(b,m,how='left')
    else:
        c = pd.merge(m,b,how='left')
    c.fillna(method='ffill', inplace=True)
    c = c.sort_values(by='trade_date')
    c['trade_date'] = pd.to_datetime(c['trade_date'])
    c.set_index("trade_date", inplace=True)
    c['bchg']=c.b.pct_change()*100
    c['mchg']=c.m.pct_change()*100
    c.fillna(value=0, inplace=True)
    #display(HTML(c.to_html()))
    return sum(pow(c['mchg']/100 - c['bchg']/100,2))*100

In [146]:
te = track_error(df1, df2)
print('%0.2f%%' % te)

0.52%


In [149]:
te = track_error(df1, df3)
print('%0.2f%%' % te)

1.13%
